In [8]:
import os
import sys
import numpy as np
# Important directories
code_dir = os.path.dirname(os.getcwd())
module_dir = os.path.dirname(os.path.dirname(os.getcwd()))

# Allows to import my own module
sys.path.insert(0, code_dir)

from LongstaffSchwarz.smodel import Heston

In [80]:
def integral(A=None,dF=None,F=None,axis = 0,trapez = False,cumulative = False):
    '''
    Turns an array A of length N (the function values in N points)
    and an array dF of length N-1 (the masses of the N-1 intervals)
    into an array of length N (the integral \int A dF at N points, with first entry 0)
    
    :param A: Integrand (optional, default ones, length N)
    :param dF: Integrator (optional, default ones, length N-1)
    :param F: Alternative to dF (optional, length N)
    :param trapez: Use trapezoidal rule (else left point)
    '''
    ndim = max(v.ndim for v in (A,dF,F) if v is not None)
    def broadcast(x):
        new_shape = [1]*ndim
        new_shape[axis] = -1
        return np.reshape(x,new_shape)
    if F is not None:
        assert(dF is None)
        if F.ndim<ndim:
            F = broadcast(F)
        N = F.shape[axis]
        dF = F.take(indices = range(1,N),axis = axis)-F.take(indices = range(N-1),axis = axis)
    elif dF is not None:
        if dF.ndim<ndim:
            dF = broadcast(dF)
        N = dF.shape[axis]+1
    else:
        if A.ndim<ndim:
            A = broadcast(A)
        N = A.shape[axis]
    if A is not None:
        if trapez:
            midA = (A.take(indices = range(1,N),axis = axis)+A.take(indices = range(N-1),axis = axis))/2
        else:
            if axis:
                midA = A.take(indices=range(N-1),axis=axis)
            else:
                midA = A[:-1]
        if dF is not None:
            dY = midA*dF
        else:
            dY = midA
    else:
        dY = dF
    pad_shape = list(dY.shape)
    pad_shape[axis] = 1
    pad = np.zeros(pad_shape)
    if cumulative:
        return np.concatenate((pad,np.cumsum(dY,axis = axis)),axis = axis)
    else:
        return np.sum(dY,axis = axis)
def heston(times,mu,rho,kappa,theta,xi,S0,nu0,d,M,nu_1d=True, random=np.random):
    '''
    Return M Euler-Maruyama sample paths with N time steps of (S_t,v_t), where
        (S_t,v_t) follows the Heston model of mathematical finance. 
    Currently requires times to be uniform
    :rtype: M x N x d array
    '''
    d_nu = 1 if nu_1d else d
    N = len(times)
    nu = np.zeros((M,N,d_nu))
    S = np.zeros((M,N,d))
    nu[:,0,:] = nu0
    S[:,0,:] = S0
    if 2*kappa*theta<=xi**2:
        raise ValueError('Feller condition not satisfied')
    test = np.std(np.diff(times.flatten())) 
    if test>1e-12:
        raise ValueError
    dt = times[1]-times[0]
    if d == 1:
        if np.array(rho).size ==1:
            rho = np.array([[1,rho],[rho,1]])
    chol = np.linalg.cholesky(rho)
    dW = np.sqrt(dt)*np.einsum('ij,...j',chol,random.normal(size=(M,N-1,d+d_nu)))
    for i in range(1,N):
        nu[:,i,:] = np.abs(nu[:,i-1,:] + kappa*(theta-nu[:,i-1,:])*dt+xi*np.sqrt(nu[:,i-1,:])*dW[:,i-1,d:])
    S = S0*np.exp(integral(np.sqrt(nu),dF = dW[:,:,:d],axis=1,cumulative = True)+integral(mu - 0.5*nu,F = times,axis=1,trapez=False,cumulative = True))
    return np.concatenate((S,nu),axis=-1)

In [119]:
import numpy as np
import scipy.stats as stats


class BS:

    def __init__(self, S, K, T, r, sigma, times, mu, rho, kappa, theta, xi, nu0, d, M, option='call'):
        self.S0 = S
        self.K = K
        self.T = T
        self.r = r
        self.sigma = sigma
        self.option = option
        self.times = times
        self.mu = mu
        self.rho = rho
        self.kappa = kappa
        self.theta = theta
        self.xi = xi
        self.nu0 = nu0
        self.d = d
        self.M = M

    def simulate(self):

        # S: spot price
        # K: strike price
        # T: time to maturity
        # r: interest rate
        # sigma: volatility of underlying asset

        d1 = (np.log(self.S / self.K) + (self.r + 0.5 * self.sigma ** 2) * self.T) / (self.sigma * np.sqrt(self.T))
        d2 = (np.log(self.S / self.K) + (self.r - 0.5 * self.sigma ** 2) * self.T) / (self.sigma * np.sqrt(self.T))

        if self.option == 'call':
            return self.S * stats.norm.cdf(d1, 0.0, 1.0) - self.K * np.exp(-self.r * self.T) * stats.norm.cdf(d2, 0.0, 1.0)
        if self.option == 'put':
            return self.K * np.exp(-self.r * self.T) * stats.norm.cdf(-d2, 0.0, 1.0) - self.S * stats.norm.cdf(-d1, 0.0, 1.0)
        else:
            raise NameError("Option undefined")

    def LSM(self, N=10000, paths=10000, order=2, S_defined = 'bs'):
        """
        Longstaff-Schwartz Method for pricing American options

        Arguments
        ---------

        N: int
         number of time steps
        paths: int
         number of generated paths
        order: int
         order of the polynomial for the regression
        """

        if self.option != "put":
            raise ValueError("invalid type. Set 'call' or 'put'")

        dt = self.T / (N - 1)  # time interval
        df = np.exp(-self.r * dt)  # discount factor per time time interval


        X0 = np.zeros((paths, 1))
        hes = np.zeros((N,paths,2))
        for i in range(N):
            hes[i,:,:] = heston(self.times,self.mu,self.rho,self.kappa,self.theta,self.xi,self.S0,self.nu0,self.d,self.M,nu_1d=True, random=np.random)
        S = self.S0 * np.exp(hes)

        H = np.maximum(S - self.K, 0)  # intrinsic values for put option
        V = np.zeros_like(H)  # value matrix
        V[:, -1] = H[:, -1]
        print(hes.shape)
        # Valuation by LS Method
        for t in range(N - 2, 0, -1):
            good_paths = H[:, t,0] > 0

            #S = np.array([hes[0,:,0]/self.K,hes[0,:,0]/self.K,hes[0,:,0]/self.K * hes[0,:,1]/self.theta])
            rg = np.polyfit(S[good_paths, t,0], V[good_paths, t + 1] * df, order)  # polynomial regression
            C = np.polyval(rg, S[good_paths, t])  # evaluation of regression

            exercise = np.zeros(len(good_paths), dtype=bool)
            print(H[good_paths, t,0].shape)
            exercise[good_paths] = H[good_paths, t, 0] > C

            V[exercise, t] = H[exercise, t]
            V[exercise, t + 1:] = 0
            discount_path = (V[:, t] == 0)
            V[discount_path, t] = V[discount_path, t + 1] * df

        V0 = np.mean(V[:, 1]) * df  #
        return V0

In [120]:
from matplotlib import pyplot as plt
bs = BS( S = 1, K = 1, T = 1, r = 0.05, sigma = 0.5, times = np.linspace(0,1,100), mu = 0.5, rho = -0.7, kappa = 0.5, 
        theta = 0.5, xi = 0.5, nu0 = 0.5, d = 1, M = 1, option='put')
bs.LSM(100,100)
#res = heston(np.linspace(0,1,10000),0.5,-0.7,0.5,0.5,0.5,1,1,1,1)
#plt.plot(res[0,:,1])


(100, 100, 2)
(100,)


ValueError: operands could not be broadcast together with shapes (100,) (100,2) 